# Exercises

## 7.  Train and fine-tune a decision tree for the moons dataset by following these steps:

1. Use make_moons(n_samples=10000, noise=0.4) to generate a moons dataset.

1. Use train_test_split() to split the dataset into a training set and a test set.

1. Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes.

1. Train it on the full training set using these hyperparameters, and measure your model’s performance on the test set. You should get roughly 85% to 87% accuracy.

In [24]:
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


sklearn.__version__

'1.5.1'

In [25]:
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [26]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

X, y = make_moons(n_samples=10000, noise=0.4)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

tree_clf = DecisionTreeClassifier()

param_grid = {
	'max_leaf_nodes': list(range(2, 100)),
	# 'min_samples_split': [2, 3, 4],
}

grid_search = GridSearchCV(tree_clf, param_grid, cv=5)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_score_)

{'max_leaf_nodes': 10}
0.8597142857142857


In [27]:
best_tree_clf = grid_search.best_estimator_
best_tree_clf.score(X_test, y_test)

0.8616666666666667

In [28]:
best_tree_clf.fit(X, y)
best_tree_clf.score(X, y)

0.8635

In [29]:
X_new, y_new = make_moons(n_samples=10000, noise=0.4)
best_tree_clf.score(X_new, y_new)

0.8545

## 8. Grow a forest by following these steps:

1. Continuing the previous exercise, generate 1,000 subsets of the training set,
each containing 100 instances selected randomly. Hint: you can use Scikit-
Learn’s ShuffleSplit class for this.

1. Train one decision tree on each subset, using the best hyperparameter values
found in the previous exercise. Evaluate these 1,000 decision trees on the test
set. Since they were trained on smaller sets, these decision trees will likely
perform worse than the first decision tree, achieving only about 80% accuracy.

1. Now comes the magic. For each test set instance, generate the predictions of
the 1,000 decision trees, and keep only the most frequent prediction (you can
use SciPy’s mode() function for this). This approach gives you majority-vote
predictions over the test set.

1. Evaluate these predictions on the test set: you should obtain a slightly higher
accuracy than your first model (about 0.5 to 1.5% higher). Congratulations,
you have trained a random forest classifier!

In [43]:
from sklearn.model_selection import ShuffleSplit
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

X, y = make_moons(n_samples=11000, noise=0.4)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

n_trees = 1000

forest = [clone(best_tree_clf) for _ in range(n_trees)]

splits = ShuffleSplit(n_splits=n_trees, test_size=0.1)

for i, (train_index, test_index) in enumerate(splits.split(X_train)):
	forest[i].fit(X[train_index], y[train_index])

In [44]:
from scipy.stats import mode

def forest_predict(forest, X):
	predictions = np.array([tree.predict(X) for tree in forest])
	return mode(predictions)

X_new, y_new = make_moons(n_samples=1000, noise=0.4)

y_pred = forest_predict(forest, X_test)
y_pred_new = forest_predict(forest, X_new)

In [46]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred[0].flatten()), accuracy_score(y_new, y_pred_new[0].flatten())

(0.8557575757575757, 0.863)